## Complétion de matrices symétriques semi-définie positive


Soient $A\in \mathcal{S}_n(\mathbb{R})$ semi-définie positive. On suppose ne connaitre qu'un certain nombre d'entrées de cette matrice : $\forall (i,j)\in\Omega, \quad A_{i,j} $ est connu.

On cherche à trouver les données manquantes (complétion de matrice). On modélise ce problème par 

$$\hspace{5cm} (\mathcal{E})\quad \text{ Trouver }\delta \in \mathcal{S}_n^+(\mathbb{R})  \text{t.q.}  \forall (i,j)\in\Omega, \quad \delta_{i,j}=A_{i,j}.$$

On cherche ainsi  $$\delta \in \mathcal{C}=\mathcal{C}_1\bigcap\mathcal{C}_2,$$ avec $$\mathcal{C}_1=\mathcal{S}_n^+(\mathbb{R}), \quad \mathcal{C}_2=\left\{\delta \in \mathcal{M}_n(\mathbb{R}) \text{ t.q. } \forall (i,j)\in\Omega,  \delta_{i,j}=A_{i,j}\right\}.$$ 

**Question 1 :** Justifier que ces deux parties sont convexes, fermées et non vides.


On définit pour cela le problème d'optimisation suivant : 
$$\hspace{5cm} (\mathcal{P})\quad \min_{\delta \in \mathcal{M}_n(\mathbb{R})} f(\delta)=\max(d(\delta,\mathcal{C}_1),d(\delta,\mathcal{C}_2))$$

avec $d(\delta,\mathcal{C}_i)$ la distance de $\delta$ à $\mathcal{C}_i$.

**Question 2 :** On choisit de munir $\mathcal{M}_n(\mathbb{R})$ du produit scalaire $<X,Y>=tr(XY^T)$. On a alors $d(\delta,\mathcal{C}_i)=\Vert \delta - \Pi_{\mathcal{C}_i}(\delta)\Vert_F$, avec $\Vert \Vert_F$ la norme de Frobenius, et $\Pi_{\mathcal{C}_i}(\delta)$ le projeté de $\delta$ sur $\mathcal{C}_i$. Donner l'expression analytique des $\Pi_{\mathcal{C}_i}(\delta)$.

**Question 3 :** Proposer un sous-gradient de $f$ en $\delta$.

**Qestion 4 :** Résoudre le problème $ (\mathcal{E})$ par l'algorithme du sous-gradient.


In [ ]:
using LinearAlgebra, Random, Plots

n=10
nd=3

#Construction de la matrice A
tmp=randn(n,2*n)
F=qr(tmp)
P=F.Q[1:n,1:n]
d=abs.(10*randn(n,))
A=P*Diagonal(d)*transpose(P)

#Indice des entrées connues
Oi=randperm(n)[1:min(nd,n)]

println(Oi)
x = randn(n);
b = A*x

In [ ]:
function π1(Δ)
    eigs, P = eigen(Δ)
    eigs = real(eigs)
    eigs = [max(0,lambda) for lambda in eigs]
    return P * diagm(eigs) * transpose(P)
end

function π2(Δ, A, Oi, Oj)
    result = copy(Δ)
    for i in Oi
        for j in Oj
            result[i, j] = A[i, j]
        end
    end
    return result
end

function evalf(Δ,A,Oi,Oj)
    d1 = norm(Δ-π1(Δ)); # with p=2, for a matrix, computes Frobenius norm
    d2 = norm(Δ-π2(Δ,A,Oi,Oj));
    return max(d1,d2);
end

# Return one subgradient of f at Δ
function subgradf(Δ,A,Oi,Oj)
    eval1 = Δ-π1(Δ);
    eval2 = Δ-π2(Δ,A,Oi,Oj);
    d1 = norm(eval1);
    d2 = norm(eval2);
    
    if(max(d1,d2)==d1)
        g = eval1/d1;
    else
        g = eval2/d2;
    end
       
    return g;
end

In [ ]:
#Pkg.add("Plots")
using Plots

# Initialisation
Δ = zeros(n,n);
Δ_best = Δ;
i = 0;
fbest = 1000000;
histo_subgradient_simple =[];

lambda=1e-2;

pas=4;
itermax_subgradient_simple=50;

Δ_k = Δ;
while i < itermax_subgradient_simple
    i = i + 1;
    gk = subgradf(Δ_k, b, x_tilde);
    #print(norm(gk,2),"\n")
    
    if pas == 1
        Δ_k -= 1e-2*gk;
    elseif pas == 2
        Δ_k -= 1/(100i) * gk;
    elseif pas == 3
        Δ_k -= 1/(norm(gk,2)*sqrt(i)) * gk;
    elseif pas == 4
        #print(evalf(Δ_k, b, x_tilde),"\n")
        Δ_k -= evalf(Δ_k, b, x_tilde)/norm(gk,2)^2 * gk;
    end
    
    if evalf(Δ_k, b, x_tilde) < fbest
        #print("hi!\n")
        fbest = evalf(Δ_k, b, x_tilde);
        Δ_best = Δ_k;
    end
    
    append!(histo_subgradient_simple, fbest);
end

plotly();

iter_subgradient_simple=1:itermax_subgradient_simple;


In [ ]:
plot(iter_subgradient_simple,histo_subgradient_simple,title="Convergence curve",label=["f"],lw=3,ann=[(55,80,"Sous-gradient simple")])  